In [8]:
import syft as sy
from syft import ActionObject
from syft.service.action.action_data_empty import ActionFileData
from collections import defaultdict

In [9]:
client = sy.login(url="http://localhost:80", email="info@openmined.org", password="changethis")

Logged into <high-side: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [10]:
client.users

In [12]:
sy.ActionObject.from_path(path="scenario_data.jsonl").send(client)

SyftWarning: You're performing an operation on high side domain which could host datasets with private information.

Would you like to proceed? [y/n]: y


SyftError: Exception calling action.set. Traceback (most recent call last):
  File "/app/syft/src/syft/node/node.py", line 918, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/syft/src/syft/service/service.py", line 339, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/syft/src/syft/service/action/action_service.py", line 96, in set
    result = self.store.set(
             ^^^^^^^^^^^^^^
AttributeError: 'MongoDocumentStore' object has no attribute 'set'

In [ ]:
scenario_obj = ActionObject.from_obj([
    sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data for i in range(2)])
scenario_files_ptr = scenario_obj.send(client)

In [56]:
# TODO: fix way we send list of files
input_obj = ActionObject.from_obj([
    sy.ActionObject.from_path("short_input.jsonl").send(client).syft_action_data for i in range(2)])
input_files_ptr = input_obj.send(client)

SyftWarning: You're performing an operation on high side domain which could host datasets with private information.

Would you like to proceed? [y/n]: y


SyftWarning: You're performing an operation on high side domain which could host datasets with private information.

Would you like to proceed? [y/n]: y


SyftWarning: You're performing an operation on high side domain which could host datasets with private information.

Would you like to proceed? [y/n]: y


In [57]:
# for line in input_files_ptr.syft_action_data[0].iter_lines():
#     print(line)

# Syft functions

In [58]:
@sy.syft_function()
def compute_document_data_overlap(scenario_file, input_files, n):
    print("starting overlap computation")

    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for scenario in light_scenarios:
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # # SETUP
    print("preparing scenarios and creating indexes")
    light_scenarios = []
    for light_scenario_json in scenario_file.iter_lines():
        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        subject_spec = light_scenario_key_dict["scenario_spec"]['args']['subject']
        light_scenario_key = subject_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
        
    stats_key_counts = defaultdict(int)
    
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    
    for input_file in input_files:
        for line in input_file.iter_lines():
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

SyftSuccess: Syft function 'compute_document_data_overlap' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [59]:
client.code.submit(compute_document_data_overlap)

SyftSuccess: User Code Submitted

In [60]:
@sy.syft_function_single_use(input_files=input_files_ptr, scenario_files=scenario_files_ptr)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


SyftSuccess: Syft function 'main_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [61]:
input_files_ptr.id

<UID: f492d3fb586e48d58e469739f31a1239>

In [62]:
client.code.request_code_execution(main_function)
client.requests[-1].approve()

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]: y
Request approved for domain high-side


SyftSuccess: Request 8db4e3cd4e3247bbac0af456992e04e0 changes applied

In [74]:
job = client.code.main_function(input_files=input_files_ptr, scenario_files=scenario_files_ptr, blocking=False)

ConnectionError: Failed to fetch metadata. Response returned with code 500

# Get results

In [64]:
job

```python
class Job:
    id: UID = 1b9bd273fa0f45cf92692b4dbfe0a3d8
    status: completed
    has_parent: False
    result: SyftError: Failed to run. 'SyftError' object has no attribute 'is_err'
    logs:

0 
JOB COMPLETED
    
```

In [34]:
job.subjobs

[]

In [32]:
job.wait()

SyftError: Failed to run. Input <class 'syft.types.uid.UID'> for input_files not in allowed {'input_files': <UID: 90d99061d1574d57adcb07393cc92023>, 'scenario_files': <UID: f07fd77bb0544df38d986be0cca44d4b>}

In [33]:
job.subjobs

[]

In [16]:
job.subjobs[0].logs()

starting overlap computation
preparing scenarios and creating indexes
computing overlap
computing overlap
done
done

starting overlap computation
preparing scenarios and creating indexes
computing overlap
computing overlap
done
done



In [17]:
results = [j.wait().get() for j in job.subjobs]

In [18]:
#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

[(defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135})),
 (defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135}))]

[(defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135})),
 (defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135}))]

Idle worker expired: b'CA3B-67F3'
Idle worker expired: b'199D-2559'
Idle worker expired: b'3E5D-6502'
Idle worker expired: b'E402-6C4E'
Idle worker expired: b'CA3B-67F3'
Idle worker expired: b'199D-2559'
Idle worker expired: b'3E5D-6502'
Idle worker expired: b'E402-6C4E'


In [19]:
results[0]

(defaultdict(set,
             {'philosophy_test_5': {'id328'}, 'philosophy_valid_5': {'id12'}}),
 defaultdict(set, {}),
 defaultdict(int,
             {'philosophy_train_5': 5,
              'philosophy_valid_5': 34,
              'philosophy_test_5': 311,
              'anatomy_train_5': 5,
              'anatomy_valid_5': 14,
              'anatomy_test_5': 135}))

(defaultdict(set,
             {'philosophy_test_5': {'id328'}, 'philosophy_valid_5': {'id12'}}),
 defaultdict(set, {}),
 defaultdict(int,
             {'philosophy_train_5': 5,
              'philosophy_valid_5': 34,
              'philosophy_test_5': 311,
              'anatomy_train_5': 5,
              'anatomy_valid_5': 14,
              'anatomy_test_5': 135}))

# Aggregate

In [19]:
stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.split('_')
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'subject': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)




In [20]:
from pprint import pprint
pprint(all_data_overlap_stats)

[{'data_overlap_stats_key': {'light_scenario_key': {'split': 'train',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': [],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 10},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'valid',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id12'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 68},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'test',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id328'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 

In [ ]:
# TODO: fix way we send list of files
scenario_obj = ActionObject.from_obj([
    sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data for i in range(2)])
scenario_files_ptr = scenario_obj.send(client)

# TODO: fix way we send list of files
input_obj = ActionObject.from_obj([
    sy.ActionObject.from_path("short_input.jsonl").send(client).syft_action_data for i in range(2)])
input_files_ptr = input_obj.send(client)

# for line in input_files_ptr.syft_action_data[0].iter_lines():
#     print(line)

# Syft functions

@sy.syft_function()
def compute_document_data_overlap(scenario_file, input_files, n):
    print("starting overlap computation")

    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for scenario in light_scenarios:
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # # SETUP
    print("preparing scenarios and creating indexes")
    light_scenarios = []
    for light_scenario_json in scenario_file.iter_lines():
        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        subject_spec = light_scenario_key_dict["scenario_spec"]['args']['subject']
        light_scenario_key = subject_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
        
    stats_key_counts = defaultdict(int)
    
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    
    for input_file in input_files:
        for line in input_file.iter_lines():
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

client.code.submit(compute_document_data_overlap)

@sy.syft_function_single_use(input_files=input_files_ptr, scenario_files=scenario_files_ptr)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


client.code.request_code_execution(main_function)
client.requests[-1].approve()

job = client.code.main_function(input_files=input_files_ptr, scenario_files=scenario_files_ptr, blocking=False)

# Get results

job

job.wait()

job.subjobs

job.subjobs[0].logs()

results = [j.wait().get() for j in job.subjobs]

#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

results[0]

# Aggregate

stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.split('_')
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'subject': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)




from pprint import pprint
pprint(all_data_overlap_stats)



In [ ]:
# %debug